The data source for this project is the National Transportation Safety Board Aviation Accident database available at 
https://app.ntsb.gov/avdata/Access/

This is a MS Access formated database composed of 20 tables. 

For the purpose of this project an initial subset of the data was filtered out of MS Access using that program's query utilities. This initial definition was meant to be broad in scope to make the dataset for the project more manageable.
This was done by avoiding to export columns that clearly would not contribute to the analysis of the problem and only selecting records pertinent to the project (i.e. commercial flight operations were left out of the export.)


The following tables were condensed and exported to csv files:
- aircraft: condensed from 93 to 34 columns and from 85337 records to 69071.

In [18]:



import pandas as pd


In [17]:
definitions = pd.read_csv('D:/cap/capstone2/data/raw/data_dic.csv',  encoding = 'Windows-1252')
print (definitions.head())

   eADMS Data  aircraft  acft_category  ct_acft_cat   AIR    Airplane  char  \
0  eADMS Data  aircraft  acft_category  ct_acft_cat  BALL     Balloon  char   
1  eADMS Data  aircraft  acft_category  ct_acft_cat  BLIM       Blimp  char   
2  eADMS Data  aircraft  acft_category  ct_acft_cat   GLI      Glider  char   
3  eADMS Data  aircraft  acft_category  ct_acft_cat  GYRO   Gyrocraft  char   
4  eADMS Data  aircraft  acft_category  ct_acft_cat  HELI  Helicopter  char   

   4.00  Aircraft Category  \
0   4.0  Aircraft Category   
1   4.0  Aircraft Category   
2   4.0  Aircraft Category   
3   4.0  Aircraft Category   
4   4.0  Aircraft Category   

  The category of the involved aircraft. In this case, the definition of aircraft category is the same as that used with respect to the certification, ratings, privileges, and limitations of airmen. Also note that there is some overlap of category and class in the available choices.  \
0  The category of the involved aircraft. In this...     

In [23]:
print(definitions[1].unique)


KeyError: 1